In [ ]:
import numpy as np
import os
import seaborn as sns

from openai import OpenAI
from math import exp
from IPython.display import display, HTML
from matplotlib import pyplot as plt

yourOPENAI_API_KEY = "Add your OPEN API key here"

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "yourOPENAI_API_KEY")


In [ ]:
def get_completion(
    messages: list[dict[str, str]],
    model: str = "gpt-4",
    max_tokens=500,
    temperature=0,
    stop=None,
    seed=123,
    tools=None,
    logprobs=None, 
    top_logprobs=None,
) -> str:
    params = {
        "model": model,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "stop": stop,
        "seed": seed,
        "logprobs": True,
        "top_logprobs": top_logprobs,
    }
    if tools:
        params["tools"] = tools

    completion = client.chat.completions.create(**params)
    return completion

In [ ]:
API_RESPONSE = get_completion(
        [{"role": "user", "content": "What is the First Name of the US President in 2023. Only responding with one single word is very important!"}],
        model="gpt-4o",
        logprobs=True,
        top_logprobs=5,
)
top_two_logprobs = API_RESPONSE.choices[0].logprobs.content[0].top_logprobs
html_content = ""
dataListUSProb = []
dataListUSName = []
for i, logprob in enumerate(top_two_logprobs, start=1):
    html_content += (
        f"<span style='color: cyan'>Output token {i}:</span> {logprob.token}, "
        f"<span style='color: darkorange'>logprobs:</span> {logprob.logprob}, "
        f"<span style='color: magenta'>linear probability:</span> {np.round(np.exp(logprob.logprob)*100,2)}%<br>"
    )
    dataListUSProb.append(np.round(np.exp(logprob.logprob)*100,4))
    dataListUSName.append(logprob.token)
display(HTML(html_content))
print("\n")

In [ ]:
API_RESPONSE = get_completion(
        [{"role": "user", "content": "Choose a colour between red, yellow, blue, golden and green. Be random! Only respond with one single word!"}],
        model="gpt-4o",
        logprobs=True,
        top_logprobs=5,
)
top_two_logprobs = API_RESPONSE.choices[0].logprobs.content[0].top_logprobs
html_content = ""
dataListColourProb = []
dataListColourName = []
for i, logprob in enumerate(top_two_logprobs, start=1):
    html_content += (
        f"<span style='color: cyan'>Output token {i}:</span> {logprob.token}, "
        f"<span style='color: darkorange'>logprobs:</span> {logprob.logprob}, "
        f"<span style='color: magenta'>linear probability:</span> {np.round(np.exp(logprob.logprob)*100,2)}%<br>"
    )
    dataListColourProb.append(np.round(np.exp(logprob.logprob)*100,2))
    dataListColourName.append(logprob.token)
display(HTML(html_content))
print("\n")

In [ ]:
sns.set_style('darkgrid')

fx, (ax1,ax2)= plt.subplots(1, 2, figsize=(10, 5))
ax1.bar(dataListColourName,dataListColourProb,width=0.1, color='black')
ax1.set_xlabel('\n Response', size=18)
ax1.set_ylabel('\n prob(Response) [%]', size=18)
ax1.set_ylim([0,100])
ax1.set_title('Choose a colour!')



ax2.bar(dataListUSName,dataListUSProb,width=0.1, color='black')
ax2.set_xlabel('\n Response', size=18)
ax2.set_ylabel(' ', size=18)
ax2.set_ylim([0,100])
ax2.set_title('What was the first name of the US President in 2023?')
ax2.set_yticklabels('')

plt.tight_layout()
